This notebook is part of the youtube video "Jet Engine Series" from the Flight Test Engineering Channel: follow this [link](https://youtube.com/@flighttestengineering)

Episode 1: Series Intro

Episdoe 2: Inlet Thermodynamics

Episode 3: Inlet Python Coding

Episode 4: Compressor Thermodynamics

Episode 5: Compressor Python Coding

Episode 6: Combustor Thermo + Python Coding

Episode 7: Turbine Thermo + Python Coding

Episode 8: *Nozzle Thermo + Python Coding* - use this notebook

# A Small Recap

Ideal Gas

Isentropic Process

* $pV^{\gamma}=k$; $p=k\rho^{\gamma}$ -> $dp=\gamma k \rho^{\gamma -1}d\rho$

Isentropic Compression

* $\frac{T_2}{T_1}=(\frac{p_2}{p_1})^{\frac{\gamma-1}{\gamma}}$  >>>or<<<   $\frac{p_2}{p_1}=(\frac{T_2}{T_1})^{\frac{\gamma}{\gamma-1}}$


Bernoulli / Compressible 

* $\frac{p_0}{p} = (1+ \frac{\gamma-1}{2} M^2)^{\frac{\gamma}{\gamma - 1}} = (1+ \frac{V^2}{2 c_p T_s})^{\frac{\gamma}{\gamma - 1}}$

Compression Efficiency

* $\eta_c = \frac{w_{c,s}}{w_c} = \frac{h_{2}^\prime-h_1}{h_2-h_1} = \frac{c_p(T_{2}^\prime-T_1)}{c_p(T_2-T_1)}$

![037](pictures/compression_efficiency.svg)


Some basic relationships

$M=\frac{V}{\sqrt{\gamma RT_{static}}}$

$\gamma R = c_p (\gamma - 1)$

# Inlet Diffuser

The conditions far away from the engine, at Station "a" (for ambient), are:

$T_{a_{static}}$, $P_{a_{static}}$, and $M_a$

If we now define *sub "0"* to mean "stagnation"...

so $T_{01}$ is the stagnation pressure at station 1;

   $T_{0a}$ is the stagnation pressure at station "a"

and $T_{a}$ is the static pressure at station "a"

#### Isentropic flow up to duct inlet

Let's assume there are no losses from Station "a" to Station 1.

$P_{01} = P_{0a} = P_{a} (1 + \frac{\gamma - 1}{2}M_a^{2})^\frac{\gamma}{\gamma -1} = P_{a} (1+ \frac{V_{a}^2}{2 c_p T_a})^{\frac{\gamma}{\gamma - 1}}$

$T_{01} = T_{0a} = T_{a} (1 + \frac{\gamma - 1}{2}M_a^{2}) = T_{a}+ (\frac{V_{a}^2}{2 c_p} )$

#### Adiabatic Duct

Inside the inlet duct, we have friction losses, but no heat transfer

$T_{02}=T_{01}=T_{0a} $ -> stagnation temperatures are the same

$=> T_{02}=T_{01}= T_{1} + \frac{V_1^2}{2c_{p1}}$

$T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $

$=> T_{1} + \frac{V_1^2}{2c_{p1}} = T_{2} + \frac{V_2^2}{2c_{p2}} $

$T_{2} = T_{1} + \frac{V_1^2}{2c_{p1}} - \frac{V_2^2}{2c_{p2}}$

#### if it was isentropic...

And if we had isentropic compression, we could write:

$\frac{p_{02}}{p_{1}} = (\frac{T_{02}^\prime}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

which can be re-written as:

$\frac{p_{02}}{p_{1}} = (1 +  \frac{T_{02}^{'} - T_{1}}{T_{1}})^{\frac{\gamma}{\gamma -1}}$



#### considering the definition of inlet efficiency...



$\eta_i=\frac{T_{02}^{'}-T_{1}}{T_{02}-T_{1}}$

$T_{02}^{'}-T_{1}=\eta_i(T_{02}-T_{1})$ 

$\frac{p_{02}}{p_{1}} = (1 +  \frac{\eta_i(T_{02}-T_{1})}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

But $T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $


$\frac{p_{02}}{p_{1}} = (1 + \frac{\eta_i V_1^2}{2c_{p1}T_{1}})^{\frac{\gamma}{\gamma -1}}$

#### mass flow

$\dot{m}=\rho V A$

In [ ]:
#01 - preamble, imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import cantera as ct
import ISA_module as ISA

from engine_helper_functions import *

plt.rcParams['figure.figsize'] = [12, 7]

In [ ]:
ct.__version__

In [ ]:
#02 - engine parameters definitions

eng_param = {} # engine physical parameters will be stored in this dictionary

eng_perf = {} # engine performance parameters in this dictionary

# engine parameters
# inlet
#   physical parameters
eng_param['A1'] = 0.30 # m2 inlet area at station 1
eng_param['A2'] = 0.32 # m2 inlet exit area at face of compressor
#   performance parameters
eng_perf['eta_i'] = 0.98 # this can be upgraded to vary with Mach and m_dot

#--------------------------------------------------------------------------

# compressor
#   physical parameters
eng_param['comp_n_stages'] = 10 # number of stages in compressor
#   performance parameters
eng_perf['CPR'] = 6.1  # overall compressor pressure rise
eng_perf['eta_c'] = 0.80 # isentropic stage efficiency for compressor

#--------------------------------------------------------------------------

# combustor
#   physical parameters
#none
#   performance parameters
eng_perf['eta_b'] = 0.9 #combustion efficiency
eng_perf['dp_over_p'] = 0.06 # loss of total pressure in combustor, in percent of inlet pressure
eng_perf['max_f'] = 0.25 # maximum fuel - percent of stoichiometric - allowed in burner - burner swirl/hot limited
eng_perf['min_f'] = 0.125 # minimum fuel - % of stoich - allowed in burner - flame stability limited
eng_perf['V_nominal'] = 45 # [m/s] - nominal combustor flow velocity

#--------------------------------------------------------------------------

# turbine
#   physical parameters
eng_param['turb_n_stages'] = 2 # number of stages in turbine
#   performance parameters
eng_perf['eta_t'] = 0.80 # isentropic stage efficiency for turbine
eng_perf['mech_loss'] = 0.99 # mechanical loss on drivetrain from turbine to compressor

#---------------------------------------------------------------------------

# nozzle
#   physical parameters
eng_param['A8'] = 0.27 # nozzle area in m2
#   performance parameters
eng_perf['eta_noz'] = 0.8 # nozzle losses due to friction

In [ ]:
#03 - inlet iterative function

def iterate_inlet(mdot:float, 
                  A:float, 
                  gas_in:ct.Solution, 
                  eta_i:float, 
                  M_in:float, 
                  gas_out:ct.Solution)->(float, bool):
    '''
    This function iterates the gas velocity at inlet exit until convergence
    
    inputs
    mdot : mass flow, in kg/s
    A     : area, in m2
    gas_in: Cantera solution object with gas at entrance of inlet
    T_in  : stagnation temperature of gas at entrance of inlet, in K
    p_in  : stagnation pressure of gas at entrance of inlet, in Pascals
    eta_i : inlet efficiency
    M_in  : Mach number for gas at entrance of inlet
    gas_out: Cantera solution object with gas at exit of inlet
    
    returns
    M_out : Mach number for gas at exit of inlet. Zero if no convergence reached
    convergence: True if converged, False if not
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    # loop control
    tol = 0.01 # tolerance to check for convergence
    max_iter = 100 # maximum number of iterations
    converged = False # keeps track of convergence
    n_iter = 0 #iteration counter
    
    # calculated input gas properties
    V_in = M_in * get_a(gas_in)
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in)
    p_0in = get_p(gas_in.P, gamma_in, M_in)

    # initial guess
    T_0out = T_0in
    V_out_guess = mdot / (gas_in.density * A)
    gamma_out = gamma_in
    
    
    while not converged and n_iter <= max_iter:
        
        # calc properties using current guess
        
        T_out = gas_in.T + (V_in**2 / (2 * gas_in.cp) - V_out_guess**2 / (2 * gas_out.cp))
        p_0out = gas_in.P * (1 + eta_i * V_in**2 / (2 * gas_in.cp * gas_in.T))**(gamma_in / (gamma_in - 1))
        p_out = p_0out * (T_out / T_0out)**(gamma_out / (gamma_out - 1))
              
        # update gas to get new properties (especially density)
        gas_out.TP = T_out, p_out
        gamma_out = get_gamma(gas_out)
        
        # update velocity calculation with new gas properties
        V_out = V_out_guess
        V_out_guess = mdot / (gas_out.density * A)
        
        # check for convergnece
        if abs(V_out - V_out_guess) < tol:
            print(f'inlet finished, converged, niter={n_iter}')
            converged = True
            M_out = V_out / get_a(gas_out)
        elif n_iter < max_iter:
            n_iter += 1
        else:
            M_out = 0
            print(f'inlet finished, NOT converged, niter={n_iter}')
            
    return M_out, converged            

# COMPRESSOR

## compressor efficiency...



$\eta_c=\frac{T_{03}^{'}-T_{02}}{T_{03}-T_{02}}$

$T_{03}-T_{02}=\frac{1}{\eta_c}(T_{03}^{'}-T_{02})=\frac{T_{02}}{\eta_c}(\frac{T_{03}^{'}}{T_{02}}-1)=\frac{T_{02}}{\eta_c}[(\frac{p_{03}}{p_{02}})^{\frac{\gamma-1}{\gamma}}-1]$

$T_{03}=T_{02}+\frac{T_{02}}{\eta_c}[(\frac{p_{03}}{p_{02}})^{\frac{\gamma-1}{\gamma}}-1]$


Multi-stage compressors

$(CR_{stage})^{n}=CPR$

$CR_{stage}=CPR^{\frac{1}{n}}$

In [ ]:
#08 - multi-stage compressor - "constant" temp rise

def multi_stage_compressor(gas_in:ct.Solution, 
                          n_stages:float, 
                          CPR:float, 
                          eta_c:float, 
                          M_in:float,
                          gas_out:ct.Solution):
    '''
    This function calculates the output properties of a multi-stage axial compressor
    
    inputs
    gas_in    : Cantera solution object with gas at entrance of inlet
    n_stages  : number of stages in axial compressor
    CPR       : overall compression ratio
    eta_c     : stage isentropic efficiency
    M_in      : Mach number for gas at entrance of inlet
    gas_out   : Cantera solution object with gas at exit of inlet
    
    returns
    T_out, p_out : list with static temperatures, static pressures of each stage
    convergence  : True if converged, False if not
    compressor_work : specific work used to compress gas [in kJ/kg]
    
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    
    # calculated input gas properties
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in) # stagnation
    p_0in = get_p(gas_in.P, gamma_in, M_in) # stagnation
    
    # pressure ratio per stage
    CR_stage = CPR**(1 / n_stages)
    
    
    # gradually shift pressure towards the initial stages
    stage_multiplier = np.ones(n_stages)
    shift = 0.001 # overall shift amount, per iteration
    shifter = np.ones(n_stages)
    step_shift = shift / n_stages # shift step per stage
    
    center = int(n_stages / 2) # middle of vector - it does not matter if we are off by one (even length vector)
    for i in range(center):
        shifter[i] = 1 + (center - i) * step_shift # shift initial stages UP
        shifter[n_stages - i - 1] = 1 - ((center - i) * step_shift) # shift later stages DOWN
    
    # pressure rise shift loop control
    converged = False
    n_iter = 0
    max_iter = 5000
    prev_delta_t = 1000 # start with high value to trigger condition
    
    # stage properties
    stages_p_out = np.zeros(n_stages) # holds the press data for each stage
    stages_T_out = np.zeros(n_stages) # holds the temp data for each stage
    stage_gas = (ct.Solution(reaction_mechanism, phase_name)) # internal object to keep track of gas properties
    stage_gas.X = comp_air
    
    # compressor output data
    compressor_work = 0 # collector for specific work used to compress gas, for all stages, in kJ/kg

    
    # pressure rise shift loop
    while not converged and n_iter <= max_iter:

        stage_gas.TP = gas_in.T, gas_in.P
        
        # thermo stages loop
        for st_counter in range(n_stages):

            T_i = stage_gas.T # keep initial temperature for work calculation
            
            gamma = get_gamma(stage_gas)
            p0 = get_p(stage_gas.P, gamma, M_in) * CR_stage * stage_multiplier[st_counter]
            T0 = T_0in / eta_c *((p0 / p_0in)**((gamma - 1) / gamma) - 1) + T_0in

            T = get_Ts(T0, gamma, M_in)
            p = get_ps(p0, T, T0, gamma)
            stage_gas.TP = T, p # do an update on TP, with previous gamma

            gamma = get_gamma(stage_gas) # update gamma
            T = get_Ts(T0, gamma, M_in)
            p = get_ps(p0, T, T0, gamma)
            stage_gas.TP = T, p # refine TP with updated gamma
            
            # store conditions for plotting later...
            stages_p_out[st_counter] = p
            stages_T_out[st_counter] = T
            
            # store stage work
            compressor_work += stage_gas.cp * (T - T_i) # in kJ/kg

            # update for next stage
            p_0in = p0
            T_0in = T0

        # logic to account for different number of stages
        if n_stages > 2: # typical multi-stage case
            max_delta_t = np.diff(stages_T_out).max()
        elif n_stages > 1: # special case : np.diff will drop one in vector length
            max_delta_t = max(stages_T_out[1] - stages_T_out[0], stages_T_out[0] - gas_in.T)
        else: # case for 1 stage
            max_delta_t = T - gas_in.T
        

        # loop objective is to get minimum temperature difference between all stages
        # by shifting pressure rise towards initial stages
        if max_delta_t < prev_delta_t and n_iter < max_iter:

            n_iter += 1
            # clear previous data and reset inputs
            T_0in = get_T(gas_in.T, gamma_in, M_in)
            p_0in = get_p(gas_in.P, gamma_in, M_in)
            compressor_work = 0 #zero out work absorbed by compressor
            
            # increase pressure shift towards initial stages
            stage_multiplier = np.multiply(stage_multiplier, shifter)
            prev_delta_t = max_delta_t
        
        elif n_iter >= max_iter:
            print(f'compressor finished, NOT converged, niter={n_iter}, max delta T={max_delta_t:0.1f}')
            n_iter += 1
        else:
            converged = True
            print(f'compressor finished, converged, niter={n_iter}, max delta T={max_delta_t:0.1f}')
        
        
    # update gas_out to pass properties back
    gas_out.TP = T, p


    return list(zip(stages_T_out, stages_p_out)), converged, compressor_work

# COMBUSTOR

## Adiabatic flow

## Typical Values

* flow velocity is kept at 30-60 m/s

* this is an order of magnitude higher than the flame speed

* air/fuel ratio between 60:1 and 120:1

* Stoichiometric for querosene: ~15:1

* Pressure Loss: $\frac{p_{03}-p_{04}}{p_{03}}=\frac{\delta p}{p_{03}}=$ 4~7%

* Combustor (burner) Efficiency: $eta_b=\frac{theoretical}{actual}$ fuel for actual $\Delta T$

In [ ]:
#10 - combustor

def iterate_combustor(gas_in:ct.Solution, 
                      V_nominal:float, 
                      M_in:float, 
                      dp_over_p:float,                
                      gas_out:ct.Solution)->(float, bool):
    '''
    This function iterates the gas velocity at combustor exit until convergence.
    It DOES NOT react the fuel/air mixture
    
    inputs
    gas_in: Cantera solution object with gas at entrance of inlet
    v_nominal : nominal gas velocity in combustor, in m/s
    M_in  : Mach number for gas at entrance of inlet
    dp_over_p : total pressure loss, non-dimensional
    gas_out: Cantera solution object with gas at exit of inlet
    
    returns
    M_out : Mach number for gas at exit of inlet. Zero if no convergence reached
    convergence: True if converged, False if not
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    
    # loop control
    tol = 0.01 # delta pressure tolerance to check for convergence
    max_iter = 100 # maximum number of iterations
    converged = False # keeps track of convergence
    n_iter = 0 #iteration counter
    
    # calculated input gas properties
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in)
    p_0in = get_p(gas_in.P, gamma_in, M_in)

    
    # initial guess
    T_0out = T_0in
    p_0out = p_0in * (1 - dp_over_p) # here we apply the pressure loss
    T_out = gas_in.T # first guess
    p_out = gas_in.P # first guess
    
    while not converged and n_iter <= max_iter:
        
        # update gas to get new properties
        gas_out.TP = T_out, p_out
        a_out = get_a(gas_out)
        M_out = V_nominal / a_out
        gamma_out = get_gamma(gas_out)
        T_out = get_Ts(T_0out, gamma_out, M_out)
        p_out = get_ps(p_0out, T_out, T_0out, gamma_out)
    
        # check for convergence on delta p
        if abs(gas_out.P - p_out) < tol:
            print(f'combustor finished, converged, niter={n_iter}')
            converged = True
        elif n_iter < max_iter:
            n_iter += 1
        else:
            M_out = 0
            print(f'combustor finished, NOT converged, niter={n_iter}')
            
    return M_out, converged

# TURBINE

We start by calculating the specific work consumed by the compressor (sub "tc" means "turbine to compressor"):

$W_{tc}=\frac{W_c}{\eta_m}$

Remember that we already have the numeric value of $W_c$ from our compressor calculation... If we had not, then we'd need the difference in temperature and an average cp...

$W_{tc}=\frac{W_c}{\eta_m} = c_{p_{average}} \frac{T_{03} - T_{02}}{\eta_m}$

This is the specific work that the turbine needs to provide to the compressor, including mechanical losses (bearings, etc).
We will ignore accessories (oil and fuel pumps, generators, etc) for simplicity.

The specific work out of the turbine is:

$W_t=c_{p_{exhaust}} (T_{04} - T_{05})$

with turbine efficiency defined as:

$\eta_t=\frac{W_t}{W_{ts}}=\frac{T_{04}-T_{05}}{T_{04}-T_{05}^{'}}$

The energy balance between the turbine and compressor is:



$c_{p_{exhaust}} (T_{04} - T_{05}) =  W_{tc} = \frac{W_c}{\eta_m}$

$c_{p_{exhaust}}  \eta_t (T_{04} - T_{05}^{'}) = \frac{W_c}{\eta_m}$

$ T_{05}^{'} = T_{04} - \frac{\frac{W_c}{\eta_m}}{c_{p_{exhaust}}  \eta_t}$

$T_{05} = T_{04}-\eta_t(T_{04}-T_{05}^{'})$

Since, for Isentropic Compression: $\frac{T_{05}^{'}}{T_{04}}=(\frac{p_{05}}{p_{04}})^{\frac{\gamma-1}{\gamma}}$

$\frac{p_{05}}{p_{04}}=(\frac{T_{05}^{'}}{T_{04}})^{\frac{\gamma}{\gamma-1}}$

$p_{05}=p_{04}(\frac{T_{05}^{'}}{T_{04}})^{\frac{\gamma}{\gamma-1}}$



In [ ]:
#12 - multi-stage turbine

def multi_stage_turbine(gas_in:ct.Solution, 
                          W_c:float,
                          n_stages:float,  
                          eta_t:float, 
                          eta_m:float,
                          M_in:float,
                          M_out:float,
                          gas_out:ct.Solution):
    '''
    This function calculates the output properties of a multi-stage axial turbine
    
    inputs
    gas_in    : Cantera solution object with gas at entrance of inlet
    W_c       : specific work consumed by compressor [in kJ/kg]
    n_stages  : number of stages in axial compressor
    eta_t     : stage isentropic efficiency
    eta_m     : mechanical efficiency in power transmission from turbine to compressor
    M_in      : Mach number for gas at entrance of inlet
    M_out     : design Mach number for turbine stages
    gas_out   : Cantera solution object with gas at exit of inlet
    
    returns
    T_out, p_out : list with static temperatures, static pressures of each stage
    turbine_work : specific work obtained from turbine [in kJ/kg]
    
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    
    # calculated input gas properties
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in)
    p_0in = get_p(gas_in.P, gamma_in, M_in)
    

    # update properties for exit Mach number    
    T_in = get_Ts(T_0in, gamma_in, M_out)
    p_in = get_ps(p_0in, T_in, T_0in, gamma_in)
    
    
    # specific work per stage
    W_per_stage = (W_c / eta_m) / n_stages
    
    # stage properties
    stages_p_out = np.zeros(n_stages) # holds the press data for each stage
    stages_T_out = np.zeros(n_stages) # holds the temp data for each stage
    stage_gas = (ct.Solution(reaction_mechanism, phase_name)) # internal object (to function) to keep track of gas properties
    stage_gas.TPX = T_in, p_in, gas_in.X
    
    # turbine output data
    turbine_work = 0 # specific work obtained from gas expansion, sum for all stages, in kJ/kg


    # thermo stages loop
    for st_counter in range(n_stages):

        gamma = get_gamma(stage_gas)
        T_i = stage_gas.T # keep intiial temperature to calculate work later
        T_0out_prime = T_0in - (W_per_stage) / (stage_gas.cp * eta_t)
        p_0out = p_0in * (T_0out_prime / T_0in)**(gamma / (gamma - 1))
        T_0out = T_0in - eta_t * (T_0in - T_0out_prime)

        # get static properties
        T = get_Ts(T_0out, gamma, M_out)
        p = get_ps(p_0out, T, T_0out, gamma)
        stage_gas.TP = T, p # do an update on TP, with current gamma

        gamma = get_gamma(stage_gas) # update gamma
        T = get_Ts(T_0out, gamma, M_out)
        p = get_ps(p_0out, T, T_0out, gamma)
        stage_gas.TP = T, p # refine TP with updated gamma

        # store conditions for plotting later...
        stages_p_out[st_counter] = p
        stages_T_out[st_counter] = T
        
        # store stage work        
        turbine_work += stage_gas.cp * (T - T_i) # in kJ/kg

        # update for next stage
        p_0in = p_0out
        T_0in = T_0out


    # update gas_out to pass properties back
    gas_out.TP = T, p

            
    return list(zip(stages_T_out, stages_p_out)), turbine_work

# NOZZLE

Assume adiabatic flow:

$T_{05}=T_{08}$


Nozzle Efficiency


$\eta_{noz}=\frac{T_{05}-T_{8}}{T_{05}-T_{8}^{'}}$ >>> $T_{05}-T_{8}=\eta_{noz}*(T_{05}-T_{8}^{'})$

and for isentropic conditions: $\frac{T_{8}^{'}}{T_{05}}=(\frac{p_8}{p_{05}})^{\frac{\gamma-1}{\gamma}}$ >>> $T_{8}^{'}=T_{05}(\frac{p_8}{p_{05}})^{\frac{\gamma-1}{\gamma}}$


$T_{05}-T_{8}=\eta_{noz}T_{05}[1-(\frac{1}{\frac{p_{05}}{p_{8}}})^{\frac{\gamma-1}{\gamma}}]$

---

Chocked Flow

at critical pressure and above ($p_{5}>=p_c$), chocked flow occurs, the exit velocity is M=1 and $p_{8}=p_c$


$\frac{T_{08}}{T_{8}}=1+\frac{V_8^2}{2c_pT_{8}}=1+\frac{\gamma-1}{2}M_8^2$
 

and (Adiabatic flow $T_{05}=T_{08}$) >>> $\frac{T_{05}}{T_{8}}=\frac{T_{08}}{T_{8}}$ 

If we say the flow is chocked, then:

* $p_8=p_c$
* $T_8=T_c$
* $M_8=1$, so:

$\frac{T_{05}}{T_{c}}=1+\frac{\gamma-1}{2}=\frac{\gamma + 1}{2}$

and from $\eta_{noz}=\frac{T_{05}-T_{8}}{T_{05}-T_{8}^{'}}$

since $T_8=T_c$, we can write

$T_{c}^{'}=T_{05} - \frac{1}{\eta_{noz}}(T_{05}-T_{c})$

so that $p_c$ is :

$p_c=p_{05}(\frac{T_{c}^{'}}{T_{05}})^{\frac{\gamma}{\gamma -1}}=p_{05}[1-\frac{1}{\eta_{noz}}(1-\frac{T_{c}}{T_{05}})]^{\frac{\gamma}{\gamma -1}}$


and since $\frac{T_{05}}{T_{c}}=\frac{\gamma + 1}{2}$

we get

$p_8=p_c = p_{05} [1-\frac{1}{\eta_{noz}}(\frac{\gamma-1}{\gamma+1})]^{\frac{\gamma}{\gamma-1}}$

and we had

$T_{c}=T_{05}\frac{2}{\gamma + 1}$

$V = Ma = M\sqrt{\gamma R T}=1*\sqrt{\gamma R T}$

$\rho=\frac{p}{RT}$

$\dot{m}=\rho V A$

---
Unchoked Flow

up to crititcal pressure, $p_{8}=p_{ambient}$

$T_{05}-T_{8}=\eta_{noz}T_{05}[1-(\frac{1}{\frac{p_{05}}{p_{8}}})^{\frac{\gamma-1}{\gamma}}]$ --->

$T_{8}=T_{05}-\eta_{noz}T_{05}[1-(\frac{1}{\frac{p_{05}}{p_{8}}})^{\frac{\gamma-1}{\gamma}}]$

Then

$V = \sqrt{2 c_p (\Delta T)}$

$\rho=\frac{p}{RT}$

$\dot{m}=\rho V A$

---
And finally, the specific thrust is:

$F=(V-V_i)+\frac{A^{*}}{\dot{m}}(p_8-p_{ambient})$

In [ ]:
#14 - nozzle

def calc_nozzle(gas_in:ct.Solution,
                M_in:float,
                eta_noz:float, 
                p_amb:float,
                A_star:float,
                V_i:float,
                gas_out:ct.Solution):
    '''
    This function calculates the flow though a convergent nozzle
    
    inputs
    gas_in: Cantera solution object with gas at entrance of inlet
    M_in : Mach nunmer at inlet of nozzle
    eta_noz : nozzle adiabatic efficiency
    p_amb  : ambient pressure at nozzle exit, in Pascals
    A_star : nozzle exit area, in m2
    V_i  : AIRCRAFT speed = gas speed at engine entrance (inlet), in m/s 
    gas_out: Cantera solution object with gas at exit of inlet
    
    returns
    chocked : True if flow is chocked at nozzle throat
    mdot : mass flow in kg/s
    M_out : Mach number for gas at exit of inlet. Zero if no convergence reached
    F : engine specif thrust. In N*s/kg

    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''

    # entry conditions
    gamma = get_gamma(gas_in)
    R = get_R(gas_in)
    p0_in = get_p(gas_in.P, gamma, M_in)
    T0_in = get_T(gas_in.T, gamma, M_in)


    # critical condition for sonic flow at throat
    pc_ratio = 1 / (1 - (1 / eta_noz) * ((gamma - 1)/(gamma + 1)))**(gamma / (gamma - 1))

    # chocked or not?
    if p_amb <= p0_in / pc_ratio:
        chocked = True
        p = p0_in / pc_ratio # critical pressure
        T = T0_in / ((gamma + 1) / 2) # critical temperature
        V = np.sqrt(gamma * R * T) # Mach 1
        rho = p / (R * T)
        mdot = rho * V * A_star # kg/s
        F = (V - V_i) + (A_star / mdot)  * (p - p_amb) # specific thrust [N s / kg]
        M_out = 1
    else:
        chocked = False
        p = p_amb
        T = T0_in - eta_noz * T0_in * (1 - (1 / (p0_in / p_amb)**((gamma - 1) / gamma)))
        V = np.sqrt(2 * gas_in.cp * (T0_in - T)) # m/s
        rho = p / (R * T)
        mdot = rho * V * A_star # kg/s
        F = (V - V_i) # specific thrust [N s / kg] Note this is the same as [m/s]
        
    
    gas_out.TP = T, p
    
    if not chocked:
        M_out = V / get_a(gas_out)
    
    return chocked, mdot, M_out, F

In [ ]:
#04 - engine ambient/operating conditions

eng_op_con = {}
eng_op_con['throttle_pos'] = 1.0 # 1 is full throttle; .5 is idle
eng_op_con['mdot_guess'] = 20 # kg/s
eng_op_con['alt'] = 35000 # ft
eng_op_con['M_i'] = 0.8

M_i = eng_op_con['M_i'] # indicated Mach number - aircraft
V_i = ISA.M2Vt(eng_op_con['M_i'], eng_op_con['alt']) * ISA.kt2ms # true airspeed in kts
p_amb = ISA.p(eng_op_con['alt']) # static
T_amb = ISA.T(eng_op_con['alt']) # static

In [ ]:
#05 - initial stations setup

gas = {} # dictionary with Cantera Solution (gas) objects for each station
M = {} # Mach number for each station

st = ["a", 1, 2, 3, 4, 5, 8] # station numbers
station_names = {st[0]:'ambient',
                 st[1]:'inlet',
                 st[2]:'inlet @ comp. face',
                 st[3]:'after compressor',
                 st[4]:'after combustor',
                 st[5]:'after turbine',
                 st[6]:'nozzle exit'}

# see https://github.com/Cantera/cantera/blob/main/data/nDodecane_Reitz.yaml
reaction_mechanism = 'nDodecane_Reitz.yaml'
phase_name = 'nDodecane_IG' # IG = ideal gas, other option is RK = Redlich-Kwong

comp_air = 'O2:0.209, N2:0.787, CO2:0.004' # composition of air
comp_fuel = 'c12h26:1' # composition of fuel


# initialize all stations with air, at ambient conditions
for station in st:
    gas[station] = (ct.Solution(reaction_mechanism, phase_name))
    gas[station].X = comp_air
    gas[station].TP = T_amb, p_amb
        
    M[station] = M_i

In [ ]:
#06 - from Station "a" to Station 1

M_calc, conv = iterate_inlet(eng_op_con['mdot_guess'],
                             eng_param['A1'],
                             gas[st[0]],
                             1, # isentropic thus efficiency=1
                             M[st[0]],
                             gas[st[1]])

# we only assign the Mach number if we reached convergnece
if conv: M[st[1]] = M_calc

In [ ]:
# create data set for plotting

st_T = []
st_p = []
st_X = []

# get T, p, X from each station that we already calculated
for x in range(0, 2):
    st_T.append(gas[st[x]].T)
    st_p.append(gas[st[x]].P)
    st_X.append(gas[st[x]].X)

In [ ]:
print_stations(st[0:2], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#07 - from Station 1 to Station 2

M_calc, conv = iterate_inlet(eng_op_con['mdot_guess'] ,
                             eng_param['A2'],
                             gas[st[1]],
                             eng_perf['eta_i'],
                             M[st[1]],
                             gas[2])

# we only assign the Mach number if we reached convergnece
# and if we did, we now assume constant Mach throughout the machine
if conv: 
    n_st = len(st) #get number of stations
    current_st = st.index(2) #get index of current station
    for i in range(current_st, n_st):
        M[st[i]] = M_calc
else:
    print('ERROR: inlet did not converge')
    ###break

st_T.append(gas[st[2]].T)
st_p.append(gas[st[2]].P)
st_X.append(gas[st[2]].X)

In [ ]:
print_stations(st[0:3], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#09 - calculate compressor exit station

st_out, conv, compressor_work = multi_stage_compressor(gas[2], 
                                eng_param['comp_n_stages'], 
                                eng_perf['CPR'], 
                                eng_perf['eta_c'], 
                                M[st[2]], 
                                gas[st[3]])



# add T, p, X from compressor stages
for x in st_out:
    st_T.append(x[0])
    st_p.append(x[1])
    st_X.append(st_X[2]) # we just add a fixed value here because the composition in the compressor is not changing


In [ ]:
print_stations(st[0:4], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#11 - combustor

# first, set fuel mixture according to a percentage of the stoichiometric
phi = (eng_perf['max_f'] - eng_perf['min_f']) * eng_op_con['throttle_pos'] + eng_perf['min_f'] # 
gas[st[4]].set_equivalence_ratio(phi=phi, fuel=comp_fuel, oxidizer=comp_air, basis='mole') #phi=1 means stoichchiometric air-fuel ratio

# then, get the mass fraction from the mixture before we burn it
mixt_frac = gas[st[4]].mixture_fraction(fuel=comp_fuel, oxidizer=comp_air, basis='mass') #(kg fuel / (kg fuel + kg oxidizer))

# iterate combustor with "cold" fuel-air mixture
M_calc, conv = iterate_combustor(gas[st[3]], 
                                 eng_perf['V_nominal'], 
                                 M[3], 
                                 eng_perf['dp_over_p'], 
                                 gas[st[4]])
if conv: 
    M[st[4]] = M_calc
else:
    print('ERROR: combustor did not converge')
    ###break

# burn, baby burn!
gas[st[4]].equilibrate('HP')

st_T.append(gas[st[4]].T)
st_p.append(gas[st[4]].P)
st_X.append(gas[st[4]].X)


# propagate combustion gas composition downstream
for i in st[5:]:
    gas[i].TPX = st_T[-1], st_p[-1], st_X[-1]


In [ ]:
print_stations(st[0:5], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#13 - turbine

st_out, turbine_work = multi_stage_turbine(gas[st[4]], 
                          compressor_work,
                          eng_param['turb_n_stages'],  
                          eng_perf['eta_t'], 
                          eng_perf['mech_loss'],
                          M[st[4]],
                          M[st[5]],
                          gas[st[5]])

# add T, p, X from turbine stages
for x in st_out:
    st_T.append(x[0])
    st_p.append(x[1])
    st_X.append(gas[st[5]].X) # we just add a fixed value here because the composition in the compressor is not changing


In [ ]:
print_stations(st[0:6], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)